# Entrega 2 - Introducción al Procesamiento del Lenguaje Natural 2018 


Este *notebook* de Python contiene las instrucciones para la segunda entrega del curso Introducción al Procesamiento de Lenguaje Natural. En el mismo se encontrará con instrucciones en bloques de texto y bloques de código para completar. Si bien no debe modificar la estructura base del notebook, puede agregar los bloques de texto o código que considere pertinentes para aportar claridad a la entrega.

Verifique que su entorno de Python 3 contiene todas las bibliotecas necesarias. Ejecute el bloque código a continuación para importar las bibliotecas nltk, sklearn y otras que le serán de utilidad. Verifique que se importan sin errores. 

In [ ]:
import nltk
import sklearn
import os 
import json
import random

## Lectura de datos

### Corpus

Observe el corpus contenido en el directorio *restaurante-review-dataset* extraído de [1]. Note la partición en entrenamiento (train), validación (val) y evaluación (test). Ejecute el bloque a continuación para cargar el contenido del corpus en tres variable: *train*, *validation* y *test*. Se utiliza como estructura de datos una lista de pares (comentario, valor), donde comentario es una string con el comentario y valor corresponde a la string "POS" o "NEG" si el comentario es positivo o negativo, respectivamente.
 
[1] Dubiau, L., & Ale, J. M. (2013). Análisis de Sentimientos sobre un Corpus en Español: Experimentación con un Caso de Estudio. In Proceedings of the 14th Argentine Symposium on Artificial Intelligence, ASAI (pp. 36-47).


In [ ]:
def load_data(directory, val):
    data = []
    for file in os.listdir(directory):
        with open(os.path.join(directory, file)) as f:
            file_data = json.load(f)
            data += [(l,val) for l in file_data]
    return data

corpus_train_dirs = [
    ("./restaurante-review-dataset/train-neg/", "NEG"),
    ("./restaurante-review-dataset/train-pos/", "POS"),
]
corpus_val_dirs = [
    ("./restaurante-review-dataset/val-neg/", "NEG"),
    ("./restaurante-review-dataset/val-pos/", "POS"),
]
corpus_test_dirs = [
    ("./restaurante-review-dataset/test-neg/", "NEG"),
    ("./restaurante-review-dataset/test-pos/", "POS"),
]

train, validation, test = [],[],[]
for d,v in corpus_train_dirs:
    train += load_data(d, v)
for d,v in corpus_val_dirs:
    validation += load_data(d, v)
for d,v in corpus_test_dirs:
    test += load_data(d, v)

random.Random(1234).shuffle(train)
random.Random(2345).shuffle(validation)
random.Random(3456).shuffle(test)

Despliegue en pantalla la cantidad de elementos positivos, negativos y totales de cada partición del corpus.

In [ ]:
trainPOS = sum(1 for comentario in train if comentario[1] == 'POS')
trainTOT = len(train)
trainNEG = trainTOT - trainPOS

validationPOS = sum(1 for comentario in validation if comentario[1] == 'POS')
validationTOT = len(validation)
validationNEG = validationTOT - validationPOS

testPOS = sum(1 for comentario in test if comentario[1] == 'POS')
testTOT = len(test)
testNEG = testTOT - testPOS

print("En la partición 'train' hay un total de {} comentarios de los cuales {} son positivos y {} son negativos.\n".format(trainTOT, trainPOS, trainNEG))
print("En la partición 'validation' hay un total de {} comentarios de los cuales {} son positivos y {} son negativos.\n".format(validationTOT, validationPOS, validationNEG))
print("En la partición 'test' hay un total de {} comentarios de los cuales {} son positivos y {} son negativos.\n".format(testTOT, testPOS, testNEG))

### Vectores

Los *word vectors* son representaciones vectoriales de las palabras construidas a partir de grandes colecciones de texto. Junto a este *notebook* se imparte un reportorio de vectores construido a partir de un corpus en español usando *skip-gram* con *negative sampling* (SGNS)[1]

En esta sección cargará en memoria y utilizará el repertorio de vectores impartido. Además, se evaluará la cobertura del repertorio en el corpus y se estudiará las palabras del corpus no contempladas en el repertorio de vectores (*out-of-vocabulary terms*).

En el directorio *vectores* se encuentran dos archivos:

- sgns_spvectors_300.txt con la lista de palabras del repertorio
- sgns_spvectors_300.npy con la matriz que contiene cada vector

Defina un mecanismo para cargar en memoria los vectores y para obtener en tiempo eficiente el vector correspondiente a una palabra. Tenga en cuenta lo siguiente:

- puede serle útil un diccionario que a cada palabra le corresponda su índice en la matriz
- resuelva que hacer con las palabras que no están en el repertorio de vectores


[1] Mikolov, T., Sutskever, I., Chen, K., Corrado, G. S., & Dean, J. (2013). Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems (pp. 3111-3119).


In [ ]:
import numpy as np

vectors = np.load('./vectores/sgns_spvectors_300.npy')

with open('./vectores/sgns_spvectors_300.txt', 'r') as f:
    words = f.read().splitlines()

numToWord = dict(list(enumerate(words)))
wordToNum = {v: k for k, v in numToWord.items()}

def getVector(vectors, word):
    try:
        vec = vectors[wordToNum[word]]
    except KeyError:
        return [0 for _ in range(300)]
    return vec

A continuación realice pruebas con los vectores almacenados. En el siguiente bloque de código realice lo siguiente:

1. Defina una función de similitud entre vectores
2. Imprima la similitud entre los vectores de las palabras de ejemplo
3. Defina un conjunto de pares de palabras del repertorio de vectores (*pares_estudiante*)
4. Imprima la similitud de los vectores de las palabras definidas en el paso anterior
5. Imprima el vector correspondiente a una palabra que no se encuentre en el repertorio de vectores

In [ ]:
from scipy.spatial.distance import cosine

def similarity(vector1, vector2):
    return 1 - cosine(vector1, vector2)

pares = [
    ('bueno','excelente'),
    ('bueno','buena'),
    ('bueno','malo'),
    ('malo','espantoso'),
    ('comida', 'ambiente'),
    ('comida', 'bebida'),
    ('comida', 'postre'),
    ('comida', 'sabor'),
    ('servicio', 'comida'),
    ('servicio', 'ambiente'),
    ('ambiente', 'calor'),
    ('frío', 'calor'),
]

print("Conjunto 'pares':\n")

for pair in pares:
    vector0 = getVector(vectors, pair[0])
    vector1 = getVector(vectors, pair[1])
    print("La similitud entre '{}' y '{}' es {}.".format(pair[0], pair[1], similarity(vector0, vector1)))

print("\nConjunto 'pares_estudiante':\n")

pares_estudiante = [
    ('bodka', 'vodka'),
    ('manuela', 'marea'),
] 

for pair in pares_estudiante:
    vector0 = getVector(vectors, pair[0])
    vector1 = getVector(vectors, pair[1])
    print("La similitud entre '{}' y '{}' es {}.".format(pair[0], pair[1], similarity(vector0, vector1)))

¿Qué observa en los resultados obtenidos?

**Respuesta:**

COMPLETAR CON LA RESPUESTA

.

En el bloque de código a continuación realice lo siguiente:
- Separe el corpus en palabras con nltk.wordpunct_tokenize (todas la partes: train, validacion y test)
- Convierta las palabras a minúsculas dado que el repertorio de vectores está en minúsculas
- Almacene las palabras resultantes en una variable llamada *vocabulario*. Considere una estructura adecuada para no tener palabras repetidas.
- Despliegue en pantalla la cantidad de palabras de *vocabulario*

In [ ]:
vocabulario = set()

for comment in train + validation + test:
    tokenizedComment = nltk.wordpunct_tokenize(comment[0].lower())
    for word in tokenizedComment:
        vocabulario.add(word)

print(len(vocabulario))


Construya los siguientes dos conjuntos:

1. Palabras de *vocabulario* que tienen un vector asociado en el repertorio de vectores.
2. Palabras de *vocabulario* que **no** tienen un vector en el repertorio de vectores.

Imprima la cantidad de palabras de cada conjunto. Imprima además un muestreo de las palabras del conjunto 2.



In [ ]:
hasVector = set()
hasVectorNot = set()

for word in vocabulario:
    try:
        vec = vectors[wordToNum[word]]
    except KeyError:
        hasVectorNot.add(word)
    hasVector.add(word)

for word in enumerate(list(hasVectorNot)):
    print(word[1])
    if (word[0] == 20): break;
    


¿Qué observa en el muestreo de palabras que no tienen un vector asociado (conjunto 2)?


**Respuesta:**

COMPLETAR CON LA RESPUESTA


.

## Representación vectorial de la oración

### Bolsa de Palabras

Realice una representación de bolsa de palabras con *stemming* para los comentarios del corpus considerando únicamente los conjuntos de entrenamiento y validación. Utilice la clase *sklearn.CountVectorizer* con una configuración de parámetros con considere adecuada. Esta representación será utilizada posteriormente para realizar clasificiación supervisada.

**Sugerencia:** Utilice el parámetro *min_df* y *max_df* para reducir la dimensión del vector de la bolsa de palabras. Se sugiere que la dimensión de la representación sea menor a 500.

En el siguiente bloque de código realice lo siguiente:

- Despliegue la representacion (bow) de la oración presentada como ejemplo
- Despliegue además la cantidad de palabras de la bolsa de palabras

In [ ]:
oracion_ej = 'Muy pero muy buena rica la comida y muy ricas tartas'

# COMPLETE A PARTIR DE AQUI


Ejecute el bloque de código a continuación para definir la funcion *imprimir_tiempo*. Esta función despliega en pantalla el tiempo transcurrido a partir del timestamp pasado como parámetro. Si desea puede utilizarla para medir el tiempo de sus ejecuciones.

In [ ]:
import time 

def imprimir_tiempo(ts):
    print("--- %s mins ---" % (float(time.time() - ts)/60))

Implemente la función *data2Xy_bow* cuyo encabezado se presenta en el bloque de código a continuación. La función transforma en vectores a los comentarios con la bolsa de palabras almacenándolos en X y sus etiquetas en y.

In [ ]:
def data2Xy_bow(bow, data):
    X = ...
    y = ...
    return X,y


Considere la lista de pares de comentarios *lista_comentarios*. En la lista *lista_comentarios_estudiante* escriba pares de comentarios que considere pertinentes para ver su similitud según el *bow* definido. Imprima en pantalla los comentarios de ambas listas junto a la similaridad obtenida según la representación de *bow*.

In [ ]:
lista_comentarios = [
    ('muy rica la comida, buenas pizzas', 'excelente pizza la salsa estaba muy rica'),
    ('no me gustó para nada. mala atención', 'me pareció todo bastante malo'),
    ('que buen servicio, hay que volver', 'excelente todo, me verán seguido por ahí'),
    ('las tartas no me gustaron', 'muy bueno servicio'),
]

lista_comentarios_estudiante = [
    # ...
]

# COMPLETE A PARTIR DE AQUI



### Centroide de vectores

En esta parte se representará cada comentario como el centroide de los vectores de las palabras que lo forman. Se pide implementar la función *txt2vec* que dado un comentario y el repertorio de vectores calcula el promedio de los vectores de las palabras del comentario.

In [ ]:
def txt2vec(vectores, comentario):
    pass
            

Implemente la función *data2Xy_vec* cuyo encabezado se presenta en el bloque de código a continuación. La función transforma a los comentarios en su representación de centroide de vectores (las filas de X) y sus etiquetas son las componente del vector y.

In [ ]:
def data2Xy_vec(vec, data):
    X = ...
    y = ...
    return X, y
    

Despliegue en pantalla los pares de comentarios de *lista_comentarios* y *lista_comentarios_estudiante* junto con la similitud de sus representaciones de centroide de vectores.

In [ ]:
for c1,c2 in lista_comentarios:
    pass
    # ...

# COMPLETAR


Comente los resultados de similitud de oraciones obtenidos con las representaciones de bolsa de palabras y las de centroide de vectores.

**Respuesta:**

COMPLETAR CON LA RESPUESTA


.

## Análisis de Sentimiento

### Support Vector Machine

En esta sección entrene un clasificador *SVM* de *sklearn* para ambas representaciones (centroide de vectores y bolsa de palabras). Busque una configuración de hiperparámetros adecuada utilizando como referencia el conjunto de validación. Utilice para comparar resultados la medida *accuracy*.

Despliegue los resultados obtenidos con *SVM* para ambas representaciones.

### Feed Fowrward Neural Networks

En esta sección utilizará un clasificador de red neuronal *feed forward* (*multilayer perceptron* - MLP) para realizar la clasificación de sentimiento de los comentarios considerando ambas representaciones: centroide de vectores y bolsa de palabras. Busque una configuración de hiperparámetros adecuada tomando como referencia la medida de *accuracy* obtenida en el conjunto de validación. 

Despliegue los mejores resultados obtenidos para ambas representaciones.

### Mejores Resultados en Validación

Analice los resultados obtenidos con cada clasificador y cada representación en el conjunto de validación. 

Tenga en cuenta que debe considerar al menos los siguientes 4 clasificadores:

- SVM con BOW
- SVM con Centroide
- MLP con BOW
- MLP con Centroide

Realice los comentarios que considere adecuados respecto a la comparación y resultados obtenidos. Si lo desea puede agregar bloques de código que muestren resultados adicionales.

**Respuesta:**

COMPLETAR CON LA RESPUESTA


.


## Evaluación y análisis de resultados

### Precision, Recall y Matriz de Confusión

Calcule la medida de *accuracy* en el conjunto de *test* para los modelos de la parte anterior. Despliegue los resultados obtenidos.

Seleccione uno de los modelos y analice sus errores en función de la matriz de confusión. Compute las medidas de *precision*, *recall* y *F*.

Despliegue algunos casos de falsos positivos y falsos negativos del clasificador seleccionado.

### Muestreo de oraciones

Utilice cada uno de los clasificadores considerados anteriormente para cada comentario de *lista_comentarios* y *lista_comentarios_estudiante*.

Despliegue en pantalla cada comentario junto con la salida obtenida por cada clasificador. Realice los comentarios que considere pertinentes.